
# Assignment 3 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 3, PROBLEM 1
Maximum Points = 8


Download the updated data folder from the course github website or just download directly the file [https://github.com/datascience-intro/1MS041-2025/blob/main/notebooks/data/smhi.csv](https://github.com/datascience-intro/1MS041-2025/blob/main/notebooks/data/smhi.csv) from the github website and put it inside your data folder, i.e. you want the path `data/smhi.csv`. The data was aquired from SMHI (Swedish Meteorological and Hydrological Institute) and constitutes per hour measurements of wind in the Uppsala Aut station. The data consists of windspeed and direction. Your goal is to load the data and work with it a bit. The code you produce should load the file as it is, please do not alter the file as the autograder will only have access to the original file.

The file information is in Swedish so you need to use some translation service, for instance `Google translate` or ChatGPT.

1. [2p] Load the file, for instance using the `csv` package. Put the wind-direction as a numpy array and the wind-speed as another numpy array.
2. [2p] Use the wind-direction (see [Wikipedia](https://en.wikipedia.org/wiki/Wind_direction)) which is an angle in degrees and convert it into a point on the unit circle **which is the direction the wind is blowing to** (compare to definition of radians [Wikipedia](https://en.wikipedia.org/wiki/Radian)). Store the `x_coordinate` as one array and the `y_coordinate` as another. From these coordinates, construct the wind-velocity vector.
3. [2p] Calculate the average wind velocity and convert it back to direction and compare it to just taking average of the wind direction as given in the data-file.
4. [2p] The wind velocity is a $2$-dimensional random variable, calculate the empirical covariance matrix which should be a numpy array of shape (2,2).

For you to wonder about, is it more likely for you to have headwind or not when going to the university in the morning.

In [16]:
import csv
import numpy as np
extra_info = []
with open('data/smhi.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i in range(11):
        extra_info.append(next(reader))
    header = next(reader)
    data = [line for line in reader]


# wind direction is the 3rd data point and wind speed is the 5th data point
wind_dir = []
wind_speed = []

for item in data:
    splitted = item[0].split(";")
    wind_dir.append(splitted[2])
    wind_speed.append(splitted[4])

problem1_wind_direction = np.array(wind_dir).astype(int)
problem1_wind_speed = np.array(wind_speed).astype(float)

print(problem1_wind_direction)
print(problem1_wind_speed)

[348  43 185 ... 269 281 252]
[0.6 0.2 0.1 ... 1.6 1.4 1.4]


In [17]:

# The wind direction is given as a compass direction in degrees (0-360)
# convert it to x and y coordinates using the standard mathematical convention
x_coordinates = []
y_coordinates = []

for direction in problem1_wind_direction:
    math_dir = -90 - direction
    radians = math_dir * np.pi / 180
    x_coordinates.append(np.cos(radians))
    y_coordinates.append(np.sin(radians))
    
    
problem1_wind_direction_x_coordinate = np.array(x_coordinates)
problem1_wind_direction_y_coordinate = np.array(y_coordinates)

x_velocity = []
y_velocity = []

for x_dir, y_dir, speed in zip(problem1_wind_direction_x_coordinate, problem1_wind_direction_y_coordinate,problem1_wind_speed):
    x_velocity.append(x_dir*speed)
    y_velocity.append(y_dir*speed)


problem1_wind_velocity_x_coordinate = np.array(x_velocity)
problem1_wind_velocity_y_coordinate = np.array(y_velocity)

print(problem1_wind_direction_x_coordinate, problem1_wind_direction_y_coordinate)
print(problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate)

[ 0.20791169 -0.68199836  0.08715574 ...  0.9998477   0.98162718
  0.95105652] [-0.9781476  -0.7313537   0.9961947  ...  0.01745241 -0.190809
  0.30901699]
[ 0.12474701 -0.13639967  0.00871557 ...  1.59975631  1.37427806
  1.33147912] [-0.58688856 -0.14627074  0.09961947 ...  0.02792385 -0.26713259
  0.43262379]


In [18]:

# Put the average wind velocity x and y coordinates here in these variables
problem1_average_wind_velocity_x_coordinate = np.average(problem1_wind_velocity_x_coordinate)
problem1_average_wind_velocity_y_coordinate = np.average(problem1_wind_velocity_y_coordinate)

avg_radian = np.arctan2(problem1_average_wind_velocity_y_coordinate, problem1_average_wind_velocity_x_coordinate)
avg_degree = avg_radian * 180 / np.pi

# First calculate the angle of the average wind velocity vector in degrees
problem1_average_wind_velocity_angle_degrees = avg_degree
# Then calculate the average angle of the wind direction in degrees (using the wind direction in the data)
problem1_average_wind_direction_angle_degrees = np.average(problem1_wind_direction)

print(problem1_average_wind_velocity_angle_degrees, problem1_average_wind_direction_angle_degrees)

# Finally, are they the same? Answer as a boolean value (True or False)
problem1_same_angle = False

69.87353002120086 192.281280627246


In [19]:
cov_matrix = np.cov(problem1_wind_velocity_x_coordinate, problem1_wind_velocity_y_coordinate)
problem1_wind_velocity_covariance_matrix = cov_matrix
print(problem1_wind_velocity_covariance_matrix)

[[1.89213988 0.06366291]
 [0.06366291 2.45331787]]


---
## Assignment 3, PROBLEM 2
Maximum Points = 8


For this problem you will need the [pandas](https://pandas.pydata.org/) package and the [sklearn](https://scikit-learn.org/stable/) package. Inside the `data` folder from the course website you will find a file called `indoor_train.csv`, this file includes a bunch of positions in (X,Y,Z) and also a location number. The idea is to assign a room number (Location) to the coordinates (X,Y,Z).

1. [2p] Take the data in the file `indoor_train.csv` and load it using pandas into a dataframe `df_train`
2. [3p] From this dataframe `df_train`, create two numpy arrays, one `Xtrain` and `Ytrain`, they should have sizes `(1154,3)` and `(1154,)` respectively. Their `dtype` should be `float64` and `int64` respectively.
3. [3p] Train a Support Vector Classifier, `sklearn.svc.SVC`, on `Xtrain, Ytrain` with `kernel='linear'` and name the trained model `svc_train`.

To mimic how [kaggle](https://www.kaggle.com/) works, the Autograder has access to a hidden test-set and will test your fitted model.

In [22]:
import pandas as pd
from sklearn.svm import SVC
df_train = pd.read_csv('data/indoor_train.csv')
print(df_train.shape)

(1154, 4)


In [23]:

Xtrain = np.array(df_train.iloc[:, :3])
Ytrain = np.array(df_train.iloc[:, -1])
print(Xtrain.shape, Ytrain.shape)
print(Xtrain.dtype, Ytrain.dtype)

(1154, 3) (1154,)
float64 int64


In [24]:

svc_train = SVC(kernel='linear')
svc_train.fit(Xtrain, Ytrain)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


---
## Assignment 3, PROBLEM 3
Maximum Points = 8


Let us build a proportional model ($\mathbb{P}(Y=1 \mid X) = G(\beta_0+\beta \cdot X)$ where $G$ is the logistic function) for the spam vs not spam data. Here we assume that the features are presence vs not presence of a word, let $X_1,X_2,X_3$ denote the presence (1) or absence (0) of the words $("free", "prize", "win")$.

1. [2p] Load the file `data/spam.csv` and create two numpy arrays, `problem3_X` which has shape **(n_texts,3)** where each feature in `problem3_X` corresponds to $X_1,X_2,X_3$ from above, `problem3_Y` which has shape **(n_texts,)** and consists of a $1$ if the email is spam and $0$ if it is not. Split this data into a train-calibration-test sets where we have the split $40\%$, $20\%$, $40\%$, put this data in the designated variables in the code cell.

2. [2p] Follow the calculation from the lecture notes where we derive the logistic regression and implement the final loss function inside the class `ProportionalSpam`. You can use the `Test` cell to check that it gives the correct value for a test-point.

3. [2p] Train the model `problem3_ps` on the training data. The goal is to calibrate the probabilities output from the model. Start by creating a new variable `problem3_X_pred` (shape `(n_samples,1)`) which consists of the predictions of `problem3_ps` on the calibration dataset. Then train a calibration model using `sklearn.tree.DecisionTreeRegressor`, store this trained model in `problem3_calibrator`. Recall that calibration error is the following for a fixed function $f$
$$
    \sqrt{\mathbb{E}[|\mathbb{E}[Y \mid f(X)] - f(X)|^2]}.
$$

4. [2p] Use the trained model `problem3_ps` and the calibrator `problem3_calibrator` to make final predictions on the testing data, store the prediction in `problem3_final_predictions`. 

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('data\spam.csv', encoding = 'cp1252')
print(df.shape)
print(df.head(5))

df = df.iloc[:, :2]
print(df.shape)
print(df.head(5))

w1 = 'free'
w2 = 'prize'
w3 = 'win'

word_appearance = []

for entry in df['v2']:
    lowered_entry = entry.lower()
    
    X1 = 1 if w1 in lowered_entry else 0
    X2 = 1 if w1 in lowered_entry else 0
    X3 = 1 if w3 in lowered_entry else 0

    X = [X1, X2, X3]
    
    word_appearance.append(X)


problem3_X = np.array(word_appearance)
problem3_Y = np.array(np.where(df['v1'] == 'ham', 0, 1))

print(problem3_X.shape, problem3_Y.shape)

X_temp, problem3_X_calib, Y_temp, problem3_Y_calib = train_test_split(problem3_X, problem3_Y, test_size = 0.2)

problem3_X_train, problem3_X_test, problem3_Y_train, problem3_Y_test = train_test_split(X_temp, Y_temp, test_size = 0.5)

print(problem3_X_train.shape,problem3_X_calib.shape,problem3_X_test.shape,problem3_Y_train.shape,problem3_Y_calib.shape,problem3_Y_test.shape)

(5572, 5)
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
(5572, 2)
     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
(5572, 3) (5572,)
(2228, 3) (11

In [28]:


class ProportionalSpam(object):
    def __init__(self):
        self.coeffs = None
        self.result = None
    
    # define the objective/cost/loss function we want to minimise
    def loss(self,X,Y,coeffs):
        beta0 = coeffs[0]
        beta = coeffs[1:]
        power = beta0 + np.dot(X, beta)
        loss = np.sum(np.log(1+np.exp(power)) - Y*power)
        
        average_loss = loss/X.shape[0]
        
        return average_loss

    def fit(self,X,Y):
        import numpy as np
        from scipy import optimize

        #Use the f above together with an optimization method from scipy
        #to find the coefficients of the model
        opt_loss = lambda coeffs: self.loss(X,Y,coeffs)
        initial_arguments = np.zeros(shape=X.shape[1]+1)
        self.result = optimize.minimize(opt_loss, initial_arguments,method='cg')
        self.coeffs = self.result.x
    
    def predict(self,X):
        #Use the trained model to predict Y
        if (self.coeffs is not None):
            G = lambda x: np.exp(x)/(1+np.exp(x))
            return np.round(10*G(np.dot(X,self.coeffs[1:])+self.coeffs[0]))/10 # This rounding is to help you with the calibration

In [39]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

problem3_ps = ProportionalSpam()
problem3_ps.fit(problem3_X_train, problem3_Y_train)

problem3_X_pred = problem3_ps.predict(problem3_X_calib).reshape(-1,1)
print(problem3_X_pred.shape)

problem3_calibrator = DecisionTreeRegressor()
problem3_calibrator.fit(problem3_X_pred, problem3_Y_calib)

(1115, 1)


,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [47]:
initial_pred = problem3_ps.predict(problem3_X_test).reshape(-1,1)
problem3_final_predictions = problem3_calibrator.predict(initial_pred)
print(problem3_final_predictions.shape)

(2229,)


---
#### Local Test for Assignment 3, PROBLEM 3
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [ ]:
try:
    import numpy as np
    test_instance = ProportionalSpam()
    test_loss = test_instance.loss(np.array([[1,0,1],[0,1,1]]),np.array([1,0]),np.array([1.2,0.4,0.3,0.9]))
    assert (np.abs(test_loss-1.2828629432232497) < 1e-6)
    print("Your loss was correct for a test point")
except:
    print("Your loss was not correct on a test point")